# Experiment Design ex2 - CoE: textual and audio features

## Imports

In [1]:
import csv
import pandas as pd
import glob
from tqdm import tqdm
import time
from pprint import pprint

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import cross_validate

## Load data

In [2]:
def load_audio(trainpath = "./data/CoE_dataset_icpr/Dev_Set/audio_descriptors/*",
               testpath = "./data/CoE_dataset_icpr/Test_Set/audio_descriptors/*",
              trainrefpath = "./data/CoE_dataset_icpr/Dev_Set/CoeDevelopmentTrainingdata.csv",
              testrefpath = "./data/CoE_dataset_icpr/Dev_Set/CoeDevelopmentTestdata.csv"):
    """ loads all audio data and includes the target variable
    kwargs:
        trainpath: path to the folder containing all audio descriptor csv files of the train set
        testpath: path to the folder containing all audio descriptor csv files of the test set
        trainrefpath: path to the csv containing filename, movie name and the target variable for the train set
        testrefpath: path to the csv containing filename, movie name and the target variable for the test set
    """
    train = pd.DataFrame()
    test = pd.DataFrame()
    trainfiles = []
    testfiles = []

    # load training data
    for csvpath in tqdm(glob.glob(trainpath), desc='Loading audio train data'):
        trainfiles.append(csvpath.split('/')[-1].split('.csv')[0])
        tmp = pd.DataFrame(pd.read_csv(csvpath, header=None)).mean(axis=1)
        train = train.append(pd.DataFrame(tmp.values.flatten()).transpose())

    # load test data
    for csvpath in tqdm(glob.glob(testpath), desc='Loading audio test data'):
        testfiles.append(csvpath.split('/')[-1].split('.csv')[0])
        tmp = pd.DataFrame(pd.read_csv(csvpath, header=None)).mean(axis=1)
        test = test.append(pd.DataFrame(tmp.values.flatten()).transpose())

    # add filename and target variable
    train['fname'] = trainfiles
    test['fname'] = testfiles
    train = train.merge(pd.read_csv(trainrefpath)[['file_name', 'goodforairplanes']], left_on='fname', right_on='file_name').drop(columns=['file_name'])
    test = test.merge(pd.read_csv(testrefpath)[['file_name', 'goodforairplanes']], left_on='fname', right_on='file_name').drop(columns=['file_name'])
    
    # set file name as index
    train.set_index(['fname'], inplace=True)
    test.set_index(['fname'], inplace=True)
    
    # replace NAs in audio
    train = train.fillna(0)
    test = test.fillna(0)
    
    return train, test

In [3]:
def load_text():
    train = pd.read_csv("./data/CoE_dataset_icpr/Dev_Set/text_descriptors/tdf_idf_dev.csv")
    test = pd.read_csv("./data/CoE_dataset_icpr/Test_Set/text_descriptors/tdf_idf_test.csv")
    
    return train, test

In [4]:
audio_train, audio_test = load_audio()
text_train, text_test = load_text()

Loading audio test data: 100%|██████████| 223/223 [01:55<00:00,  1.93it/s]


## Learning algorithms

In [5]:
def seperate_tvar(train, test, target_var):
    """ seperate training/test set and the target variable
    kwargs:
        train: dataframe containing dependent and independent variables
        test: dataframe containing dependent and independent variables
        target_var: column name of the target variable as a string
    """
    if target_var in train.columns:
        train_y = train[target_var]
        train_x = train.drop(columns=target_var)
    if target_var in test.columns:
        test_y = test[target_var]
        test_x = test.drop(columns=target_var)
    
    return train_y, train_x, test_y, test_x

In [6]:
# dict with classifier name: time spent for fitting and classifier obejcts
classifier = {"KNN": KNeighborsClassifier(),
              "SVM": SVC(),
              "Naive bayes": GaussianNB()}

res = pd.DataFrame(columns=['Algorithm', 'Prediction', 'Precision', 'Recall', 'F-score', 'Support', 'Training time']).set_index(['Algorithm'])

for c in classifier:
    train_y, train_x, test_y, test_x = seperate_tvar(audio_train, audio_test, 'goodforairplanes')
    
    model = classifier[c]
    start_time = time.time()
    model.fit(train_x, train_y)
    dur = time.time() - start_time
    pred_y = model.predict(test_x)
    prfs = precision_recall_fscore_support(test_y, pred_y)
    res.loc[c,:] = [pred_y, prfs[0], prfs[1], prfs[2], prfs[3], dur]

res

/home/seb/.anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


,Prediction,Precision,Recall,F-score,Support,Training time
Algorithm,,,,,,
KNN,"[1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, ...","[0.6086956521739131, 0.6]","[0.5833333333333334, 0.625]","[0.5957446808510638, 0.6122448979591836]","[24, 24]",0.00129151
SVM,"[1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, ...","[0.55, 0.5357142857142857]","[0.4583333333333333, 0.625]","[0.5, 0.576923076923077]","[24, 24]",0.00169206
Naive bayes,"[0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, ...","[0.5161290322580645, 0.5294117647058824]","[0.6666666666666666, 0.375]","[0.5818181818181819, 0.4390243902439025]","[24, 24]",0.000923634
